In [4]:
import numpy as np
np.set_printoptions(suppress=True)

#import row data
row_data = np.loadtxt('sarcos_inv.csv',delimiter = ',')
np.random.seed(1)
np.random.shuffle(row_data)
X = row_data[:,:21]
Y = row_data[:,21:]

# normalize_x
X = (X - np.mean(X)) / np.std(X)

#split data
split_size = [0.6,0.2,0.2]
index_train = int(X.shape[0] * split_size[0])
index_cv = index_train + int(X.shape[0] * split_size[1])
index_test = index_cv + int(X.shape[0] * split_size[2])
X_train = X[:index_train, :]
Y_train = Y[:index_train, :]
X_test = X[index_cv:index_test, :]
Y_test = Y[index_cv:index_test, :]

In [6]:
length_scale=0.9
#   K = GaussianProcess.squared_exponential_kernel(self.X_train, self.X_train, length_scale)
dist = np.sum(X_train**2, axis=1, keepdims=True) + np.sum(X_train**2, axis=1) - 2 * np.dot(X_train, X_train.T)
K = np.exp(-(1 / (2 * length_scale**2)) * dist)
L = np.linalg.cholesky(K)
#  K_star_ = GaussianProcess.squared_exponential_kernel(X_test, self.X_train, length_scale)
dist = np.sum(X_test**2, axis=1, keepdims=True) + np.sum(X_train**2, axis=1) - 2 * np.dot(X_test, X_train.T)
K_star_ = np.exp(-(1 / (2 * length_scale**2)) * dist)
v = np.linalg.solve(L, Y_train)
w = np.linalg.solve(L.T, v)
mu = np.dot(K_star_, w)
q = np.linalg.solve(L, K_star_.T)
z = np.linalg.solve(L.T, q)
#  K__star_star = GaussianProcess.squared_exponential_kernel(X_test, X_test, length_scale) 
dist = np.sum(X_test**2, axis=1, keepdims=True) + np.sum(X_test**2, axis=1) - 2 * np.dot(X_test, X_test.T)
K__star_star = np.exp(-(1 / (2 * length_scale**2)) * dist)
K_conditioned = K__star_star - np.dot(K_star_, z)
sigma_squared = np.diag(K_conditioned)
rmse = np.sqrt(np.mean(np.square(mu - Y_test)))

MemoryError: 

In [3]:
print("Gaussian process :")
print("RMSE: {}.".format(rmse))

Gaussian process :
RMSE: 217.41887000387823.


In [ ]:

def sarcos_gaussian_processes():
    X_train, Y_train, X_cv, Y_cv, X_test, Y_test = data.get_data_split([0.6, 0.2, 0.2], normalize=True)
    print('Gaussian processes, sarcos:')
    print('\tThis will take about 15 minutes and use a lot of memory, please wait...')
    model = GaussianProcess(X_train, Y_train)
    rmse, mean, var = model.test(X_test, Y_test, length_scale=0.9, use_cholesky=True)
    print('\tRMSE: ' + str(rmse))
